In [1]:
%run auxTensor.ipynb

In [2]:
import tensorflow as tf
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

No GPU found


## 1. Generating TENSOR

In [3]:
all_releases_df = pd.read_csv('all_releases.csv')

#all_releases_df.head()

print("Generating a new dataframe without containing the last release...")
df = all_releases_df[all_releases_df['release'] != all_releases_df['release'].max()]
print("... DONE!")

df.drop(columns=['class_frequency', 'number_of_changes', 'release', 'Name', 'Kind', 'File'])

Generating a new dataframe without containing the last release...
... DONE!


,CountClassCoupled,CountDeclMethod,CountDeclMethodAll,CountClassDerived,CountLineCode,SumCyclomatic,PercentLackOfCohesion,MaxInheritanceTree,Path,will_change
0,1.0,1.0,1.0,12.0,48.0,8.0,0.0,1.0,org.apache.cassandra.CleanupHelper.CleanupHelp...,0
1,3.0,3.0,3.0,0.0,18.0,0.0,0.0,1.0,org.apache.cassandra.Util.Util.java,0
2,13.0,5.0,5.0,0.0,162.0,35.0,0.0,1.0,org.apache.cassandra.bulkloader.CassandraBulkL...,0
3,0.0,1.0,1.0,0.0,6.0,17.0,100.0,2.0,org.apache.cassandra.bulkloader.CassandraBulkL...,0
4,9.0,3.0,3.0,0.0,68.0,18.0,55.0,2.0,org.apache.cassandra.bulkloader.CassandraBulkL...,0
...,...,...,...,...,...,...,...,...,...,...
127261,0.0,14.0,32.0,0.0,79.0,19.0,71.0,3.0,org.apache.cassandra.utils.vint.EncodedDataInp...,1
127262,0.0,8.0,32.0,0.0,60.0,13.0,50.0,4.0,org.apache.cassandra.utils.vint.EncodedDataOut...,0
127263,2.0,3.0,16.0,0.0,46.0,6.0,33.0,2.0,org.apache.pig.test.MiniCluster.MiniCluster.java,1
127264,1.0,13.0,13.0,1.0,61.0,16.0,79.0,1.0,org.apache.pig.test.MiniGenericCluster.MiniGen...,0


In [4]:
listResults = {} 
for i in [2, 3, 4, 5, 6]:
    print("window_size: "+str(i))
    listResults[i] = applyTensor(i, df)
    print("... DONE!")

window_size: 2
generating tensor...
...DONE!
tensor dimensions:
tensor input X: (118733, 8, 2)
tensor input y: (118733, 2)
proportion of y labels: (array([0, 1]), array([22043, 96690]))
Splitting dataset into Train and Test sets...
Tensor X train: (83113, 2, 8)
Tensor y train: (83113, 2)
Tensor X test: (35620, 2, 8)
Tensor y test: (35620, 2)
computing weights...
... DONE!
setting stratified k-fold...
number of k: 5
... DONE!
Executing algorithm...
number of epochs: 20
number of batch: 512
Training on fold 1/5...
model dim:  (2, 8) 2
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 2, 256)            204288    
_________________________________________________________________
dropout (Dropout)            (None, 2, 256)            0         
_________________________________________________________________
gru_1 (GRU)                  (None, 2, 128)  

AttributeError: in user code:

    /Users/rogeriocarvalho/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    <ipython-input-1-fb283a009ca9>:2 auc  *
        return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

    AttributeError: module 'tensorflow' has no attribute 'py_func'


In [5]:
for i in [2, 3, 4, 5, 6]:
    print("RESULTS FROM WINDOW SIZE",i)
    lastModel = listResults[i][0]
    history_general = listResults[i][1]
    X_test = listResults[i][2]
    y_test = listResults[i][3]
    time_in_seconds = listResults[i][4]

    fractions_t = 1-y_test.sum(axis=0)/len(y_test)
    weights_t = fractions_t[y_test.argmax(axis=1)]

    output = lastModel.predict_classes(X_test)
    print(confusion_matrix(y_test.argmax(axis=1), output))
    print(classification_report(y_test.argmax(axis=1), output))
    evaluation_metrics(y_test.argmax(axis=1), output, weights_t)
    print("Time in seconds:", time_in_seconds)
    
    print("\n")

RESULTS FROM WINDOW SIZE 2
[[ 6040   593]
 [ 2899 26088]]
              precision    recall  f1-score   support

           0       0.68      0.91      0.78      6633
           1       0.98      0.90      0.94     28987

    accuracy                           0.90     35620
   macro avg       0.83      0.91      0.86     35620
weighted avg       0.92      0.90      0.91     35620

ROC_AUC: 0.9052940865359024
F1-Score: 0.9047890461243041
Precision: 0.9096399038442895
Recall: 0.8999896505329887
Accuracy: 0.9052940865359091
Time in seconds: 309.37557673454285


RESULTS FROM WINDOW SIZE 3
[[ 5905   286]
 [ 2890 24172]]
              precision    recall  f1-score   support

           0       0.67      0.95      0.79      6191
           1       0.99      0.89      0.94     27062

    accuracy                           0.90     33253
   macro avg       0.83      0.92      0.86     33253
weighted avg       0.93      0.90      0.91     33253

ROC_AUC: 0.9235060487518971
F1-Score: 0.921116033